# Assessment 2: NVIDIA Jetbot Path Following and Pest Detection
### Objective: 
The objective of this assessment is to evaluate the students' ability to apply and enhance AI models for real-world applications using the NVIDIA Jetbot platform.
### Training the Jetbot:
You are required to use the NVIDIA Jetbot to train it to follow a predefined path within a specified environment, in this case the classroom following the pink tape.
You must document the entire training process, including code, model selection, and training parameters, in a Jupyter notebook.
The Jupyter notebook should include comments explaining the rationale behind their choices and any issues they encountered.

### Inference for Pest Detection:
Using the model you have created in Assessment 1 (or a similar one), you must implement inference to detect pests.
You need to demonstrate how the Jetbot can identify and respond to pests in real-time during its path following.
The Jupyter notebook should contain code for inference and comments explaining the model's performance.

# Critical Evaluation:
You must critically evaluate the accuracy and effectiveness of your trained model for pest detection during infernece.
You must offer insightful comments on areas where improvements can be made, such as model accuracy, inference speed, and generalizability.
The Jupyter notebook should include a section for this evaluation.

# Jupyter Notebook:
The Jupyter notebook should be well-organized and include clear explanations and comments where required to demonstrate the student's knowledge and thought process.

# Video Demonstration:
You must provide a short (less than 1-minute) video demonstrating the Jetbot in action (using Xbox game bar recording or simlilar). The video should showcase the path following and pest detection aspects.

Videos go here